# testing

In [ ]:
inl __expect fn log name b a =
    inl result = fn a b
    inl result =
        result || join result
    if log |> not
    then "__expect"
    else
        inl text = $'$"{!name} / actual: %A{!a} / expected: %A{!b}"'
        text |> console.write_line
        text
    |> assert result

inl __assert_approx_eq log e b a = __expect (fun a b => abs (b - a) < (e |> optionm.defaultWith 0.00000001)) log "assert_approx_eq" b a
inl _assert_approx_eq e b a = __assert_approx_eq true e b a

inl __assert_eq log b a = __expect (=) log "assert_eq" b a
inl _assert_eq b a = __assert_eq true b a

inl __assert_eq' log b a = __expect (=.) log "assert_eq'" b a
inl _assert_eq' b a = __assert_eq' true b a

inl __assert_ne log b a = __expect (<>.) log "assert_ne" b a
inl _assert_ne b a = __assert_ne true b a

inl __assert_gt log b a = __expect (>) log "assert_gt" b a
inl _assert_gt b a = __assert_gt true b a

inl __assert_ge log b a = __expect (>=) log "assert_ge" b a
inl _assert_ge b a = __assert_ge true b a

inl __assert_lt log b a = __expect (<) log "assert_lt" b a
inl _assert_lt b a = __assert_lt true b a

inl __assert_le log b a = __expect (<=) log "assert_le" b a
inl _assert_le b a = __assert_le true b a

inl __assert_contains forall t. log (b : t) a =
    __expect
        fun a b =>
            a
            |> $'List.ofSeq'
            |> fun x => x : listm'.list' t
            |> $'List.tryFind' ((=) b)
            |> optionm'.unbox
            |> fun (x : _ t) => x <> None
        log "assert_contains" b a
inl _assert_contains b a = __assert_contains true b a

inl __assert_string_contains log b a = __expect sm'.contains log "assert_string_contains" a b
inl _assert_string_contains b a = __assert_string_contains true b a

inl _throws (fn : () -> ()) : option exn =
    inl none = None : option exn
    inl some (s : exn) = Some s
    $"try !fn (); !none with ex -> ex |> !some"

inl __assert_between log a b actual =
    inl assert_between actual (a, b) =
        __assert_ge false a actual
        __assert_le false b actual
        true
    __expect assert_between log "assert_between" (a, b) actual
inl _assert_between a b actual = __assert_between true a b actual

()



In [ ]:
//// test

1f64
|> _assert_approx_eq (Some 3) 2

let rec method0 () : unit =
    let v0 : string = "assert_approx_eq"
    let v1 : string = $"{v0} / actual: %A{1.0} / expected: %A{2.0}"
    System.Console.WriteLine v1
    ()
method0()

assert_approx_eq / actual: 1.0 / expected: 2.0


In [ ]:
//// test

"abcd"
|> _assert_contains 'b'

type [<Struct>] US0 =
    | US0_0 of f0_0 : char
    | US0_1
let rec closure0 () (v0 : char) : bool =
    let v1 : bool = 'b' = v0
    v1
and closure1 () (v0 : char) : US0 =
    US0_0(v0)
and method1 () : (char -> US0) =
    closure1()
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (string -> char list) = List.ofSeq
    let v1 : string = "abcd"
    let v2 : char list = v0 v1
    let v3 : ((char -> bool) -> (char list -> char option)) = List.tryFind
    let v4 : (char -> bool) = closure0()
    let v5 : (char list -> char option) = v3 v4
    let v6 : char option = v5 v2
    let v7 : (char -> US0) = method1()
    let v8 : US0 = US0_1
    let v9 : US0 = v6 |> Option.map v7 |> Option.defaultValue v8 
    let v11 : bool =
        match v9 with
        | US0_1 -> (* None *)
            true
        | _ ->
            false
    let v12 : bool = v11 <> true
    let v14 : bool =
        if v12 then
            true
        else
            method2(v12)
    let v15 :

In [ ]:
//// test

(dyn 1f64)
|> _assert_approx_eq (Some 3) 2

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : float = 1.0
    let v1 : float = 2.0 - v0
    let v2 : float =  -v1
    let v3 : bool = v1 >= v2
    let v4 : float =
        if v3 then
            v1
        else
            v2
    let v5 : bool = v4 < 3.0
    let v7 : bool =
        if v5 then
            true
        else
            method1(v5)
    let v8 : string = "assert_approx_eq"
    let v9 : string = $"{v8} / actual: %A{v0} / expected: %A{2.0}"
    System.Console.WriteLine v9
    let v10 : bool = v7 = false
    if v10 then
        failwith<unit> v9
method0()

assert_approx_eq / actual: 1.0 / expected: 2.0


In [ ]:
inl print_and_return x =
    $"printfn $\"print_and_return / x: {!x}\""
    x

()

